In [ ]:
import copy

# ---------- Neighbor Generator ----------
def generate_neighbors(board):
    n = len(board)
    zi, zj = next((i, j) for i in range(n) for j in range(n) if board[i][j] == 0)

    neighbors = []
    directions = [(-1, 0, "U"), (1, 0, "D"), (0, -1, "L"), (0, 1, "R")]

    for di, dj, move in directions:
        ni, nj = zi + di, zj + dj
        if 0 <= ni < n and 0 <= nj < n:
            new_board = copy.deepcopy(board)
            new_board[zi][zj], new_board[ni][nj] = new_board[ni][nj], new_board[zi][zj]
            neighbors.append((move, new_board))
    return neighbors

# ---------- Goal Check ----------
def is_goal(board, goal):
    return board == goal

# ---------- Depth Limited Search ----------
def depth_limited_search(board, goal, limit, path=None, visited=None):
    if path is None:
        path = []
    if visited is None:
        visited = set()

    board_tuple = tuple(tuple(row) for row in board)
    if board_tuple in visited:
        return None
    visited.add(board_tuple)

    if is_goal(board, goal):
        return path
    if limit == 0:
        return None

    for move, neighbor in generate_neighbors(board):
        result = depth_limited_search(neighbor, goal, limit - 1, path + [move], visited)
        if result is not None:
            return result
    return None

# ---------- Search with Increasing Depth ----------
def increasing_depth(start, goal, depth_limit):
    for depth in range(depth_limit + 1):
        result = depth_limited_search(start, goal, depth)
        if result is not None:
            return result
    return None

# ---------- Apply Moves & Write Output ----------
def write_output(solution, start, out):
    def board_to_str(b):
        return "\n".join(" ".join(str(x) for x in row) for row in b) + "\n"

    cur = [row[:] for row in start]
    out.write("Initial state:\n")
    out.write(board_to_str(cur) + "\n")

    n = len(cur)
    zi, zj = next((i, j) for i in range(n) for j in range(n) if cur[i][j] == 0)
    move_map = {'U': (-1, 0), 'D': (1, 0), 'L': (0, -1), 'R': (0, 1)}

    for move in solution:
        di, dj = move_map[move]
        ni, nj = zi + di, zj + dj
        cur[zi][zj], cur[ni][nj] = cur[ni][nj], cur[zi][zj]
        zi, zj = ni, nj
        out.write(f"After move {move}:\n")
        out.write(board_to_str(cur) + "\n")

# ---------- MAIN ----------
if __name__ == "__main__":
    with open("input.txt", "r") as f:
        content = [line for line in f.read().strip().splitlines() if line.strip()]

    idx = 0
    puzzle_no = 1
    with open("output.txt", "w") as out:   # clear old content
        out.write("")

    while idx < len(content):
        n = int(content[idx]); idx += 1

        # start board
        start = [list(map(int, content[idx + i].split())) for i in range(n)]
        idx += n

        # goal board
        goal = [list(map(int, content[idx + i].split())) for i in range(n)]
        idx += n

        # depth limit
        depth_limit = int(content[idx]); idx += 1

        solution = increasing_depth(start, goal, depth_limit)

        with open("output.txt", "a") as out:
            out.write(f"Puzzle {puzzle_no}:\n")
            if solution:
                write_output(solution, start, out)
            else:
                out.write("No solution found within given depth limit.\n\n")

        puzzle_no += 1
